In [39]:
import cv2
import os
import numpy as np
import tensorflow as tf

# Data preprocessing

In [40]:
path = 'Dataset'
categories = os.listdir(path)
label = [i for i in range(len(categories))]
label_dict = dict(zip(categories,label))
label_dict


{'without_mask': 0, 'with_mask': 1}

In [41]:
img_size = 100
data = []
label = []

for item in categories:
    folder_path = os.path.join(path, item)
    images = os.listdir(folder_path)
    
    for img_name in images:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (img_size, img_size))
        data.append(resized)
        label.append(label_dict[item])
        
    

In [42]:

data = np.array(data)/255.0
label = np.array(label)
label[1322]

1

In [43]:
data=np.reshape(data,(data.shape[0],img_size,img_size,1))

In [44]:
label=tf.keras.utils.to_categorical(label)
label[7]

array([1., 0.], dtype=float32)

# Neural Network

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,label,test_size=0.1)

In [48]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('nabin-{epoch:03d}.model',save_best_only=True)
history = model.fit(train_data,train_target,callbacks=[checkpoint],epochs=20,validation_data=(test_data, test_target))

Epoch 1/20
43/43 [==============================] - 51s 1s/step - loss: 0.5468 - accuracy: 0.7008 - val_loss: 0.5280 - val_accuracy: 0.7417
INFO:tensorflow:Assets written to: nabin-001.model\assets
Epoch 2/20
43/43 [==============================] - 53s 1s/step - loss: 0.3613 - accuracy: 0.8452 - val_loss: 0.3638 - val_accuracy: 0.8477
INFO:tensorflow:Assets written to: nabin-002.model\assets
Epoch 3/20
43/43 [==============================] - 52s 1s/step - loss: 0.2283 - accuracy: 0.9226 - val_loss: 0.2617 - val_accuracy: 0.8940
INFO:tensorflow:Assets written to: nabin-003.model\assets
Epoch 4/20
43/43 [==============================] - 52s 1s/step - loss: 0.1936 - accuracy: 0.9182 - val_loss: 0.2377 - val_accuracy: 0.9272
INFO:tensorflow:Assets written to: nabin-004.model\assets
Epoch 5/20
43/43 [==============================] - 52s 1s/step - loss: 0.1428 - accuracy: 0.9492 - val_loss: 0.2214 - val_accuracy: 0.9338
INFO:tensorflow:Assets written to: nabin-005.model\assets
Epoch 6/20

KeyboardInterrupt: 

In [50]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined